In [ ]:
!pip install librosa

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install pydub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install ffmpeg 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for ffmpeg: filename=ffmpeg-1.4-py3-none-any.whl size=6084 sha256=088e2c60b8f7ca33d1df80c20b6d755618b27292e21287d146feca41c5767332
  Stored in directory: /root/.cache/pip/wheels/64/80/6e/caa3e16deb0267c3cbfd36862058a724144e19fdb9eb03af0f
Successfully built ffmpeg


In [ ]:
!pip install soundfile

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import librosa, librosa.display
import numpy as np
import pandas as pd
import seaborn as sns
import time
import matplotlib.pyplot as plt
import glob as gb
import cv2
from tqdm import tqdm
from pydub import AudioSegment
import soundfile as sf
import subprocess
%matplotlib inline
from numpy import dot 
from numpy.linalg import norm 
import numpy as np
import sklearn.preprocessing 
import subprocess
from sklearn.preprocessing import StandardScaler
import cv2
from tensorflow.keras.models import load_model
from scipy.stats import mode

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data = pd.read_csv('/content/drive/Shareddrives/2022-1 KUBIG 음악추천시스템/0511_features_sorted.csv')
metadata = pd.read_csv('/content/drive/Shareddrives/2022-1 KUBIG 음악추천시스템/0511_metadata_sorted.csv')

In [ ]:
save_path = '/content/drive/Shareddrives/2022-1 KUBIG 음악추천시스템/'
audio_path = '/content/drive/Shareddrives/2022-1 KUBIG 음악추천시스템/'

In [ ]:
import warnings

def mp3towav(title):
  audio_file = audio_path + '/' + title +'.mp3'
  save_file = save_path + '/' + title + '.wav'
  subprocess.call(['ffmpeg', '-i', audio_file, save_file])

  print('Converting ',title,'.mp3 to wav...')
  warnings.filterwarnings("ignore") 
  
  return audio_file, save_file

In [ ]:
def trim_audio_data(audio_file, save_file, title):
    sr = 44100
    sec = 30
    
    y, sr = librosa.load(audio_file, sr = sr)
    ny = y[sr*60:sr*90]
    sf.write(save_file, ny, sr)
    
    print('Trimming ',title,'...')

    return ny, sr

In [ ]:
def extract_feature_means(y, sr, title) -> pd.DataFrame:
    # config settings
    # number_of_mfcc = c.NUMBER_OF_MFCC

    # 1. Importing 1 file
    
    D = np.abs(librosa.stft(y, n_fft=2048, hop_length=512)) #n_fft : window size / 이 때, 음성의 길이를 얼마만큼으로 자를 것인가? 를 window라고 부른다.
    DB = librosa.amplitude_to_db(D, ref=np.max)
    
    S = librosa.feature.melspectrogram(y, sr=sr)
    S_DB = librosa.amplitude_to_db(S, ref=np.max)
    
    tempo , _ = librosa.beat.beat_track(y,sr=sr) 
    
    zero_crossings = librosa.zero_crossings(y, pad=False)
    
    y_harm, y_perc = librosa.effects.hpss(y)
    
    spectral_centroids = librosa.feature.spectral_centroid(y, sr=sr)[0]
    
    spectral_rolloff = librosa.feature.spectral_rolloff(y, sr=sr)[0]
    
    
    mfccs = librosa.feature.mfcc(y, sr=sr)
    mfccs = sklearn.preprocessing.minmax_scale(mfccs,axis=1)
    
    chromagram = librosa.feature.chroma_stft(y, sr=sr, hop_length=512)
    
    spectral_rolloff = librosa.feature.spectral_rolloff(y, sr=sr)[0]

    spectral_bandwidth_2 = librosa.feature.spectral_bandwidth(y, sr=sr)[0]
    spectral_bandwidth_3 = librosa.feature.spectral_bandwidth(y, sr=sr, p=3)[0]
    spectral_bandwidth_4 = librosa.feature.spectral_bandwidth(y, sr=sr, p=4)[0]

    audio_features = {
        "zero_crossing_rate": np.mean(librosa.feature.zero_crossing_rate(y)[0]),
        "zero_crossings": np.sum(librosa.zero_crossings(y, pad=False)),
        "spectrogram": np.mean(DB[0]),
        #"mel_spectrogram": np.mean(S_DB[0]),
        "harmonics": np.mean(y_harm),
        "perceptual_shock_wave": np.mean(y_perc),
        "spectral_centroids": np.mean(spectral_centroids),
        #"spectral_centroids_delta": np.mean(spectral_centroids_delta),
        #"spectral_centroids_accelerate": np.mean(spectral_centroids_accelerate),
        "chroma1": np.mean(chromagram[0]),
        "chroma2": np.mean(chromagram[1]),
        "chroma3": np.mean(chromagram[2]),
        "chroma4": np.mean(chromagram[3]),
        "chroma5": np.mean(chromagram[4]),
        "chroma6": np.mean(chromagram[5]),
        "chroma7": np.mean(chromagram[6]),
        "chroma8": np.mean(chromagram[7]),
        "chroma9": np.mean(chromagram[8]),
        "chroma10": np.mean(chromagram[9]),
        "chroma11": np.mean(chromagram[10]),
        "chroma12": np.mean(chromagram[11]),
        "tempo_bpm": tempo,
        "spectral_rolloff": np.mean(spectral_rolloff),
        #"spectral_flux": np.mean(onset_env),
        "spectral_bandwidth_2": np.mean(spectral_bandwidth_2),
        "spectral_bandwidth_3": np.mean(spectral_bandwidth_3),
        "spectral_bandwidth_4": np.mean(spectral_bandwidth_4),
    }

 
    df = pd.DataFrame.from_records(data=[audio_features])
    df = np.array(df)
    #df = pd.merge(df, mfcc_df, on='file_name')

    print('Extracting Features of ',title,'...')

    return df

In [ ]:
def cos_sim(a, b): 
    return dot(a, b)/(norm(a)*norm(b))

In [ ]:
def rec_music(new,genre_num):

  cosdf = pd.DataFrame(columns=['idx','cos_sim']) 
  if genre_num == 0 :
      for i in range(genre_num * 200 +1,(genre_num+1) * 200 ):
        sim = cos_sim(new,data.loc[i])
        cosdf= cosdf.append({'idx': i,'cos_sim':sim}, ignore_index=True)
  else:
      for i in range((genre_num+1) * 200 +1,(genre_num+2) * 200 ):
        sim = cos_sim(new,data.loc[i])
        cosdf= cosdf.append({'idx': i,'cos_sim':sim}, ignore_index=True)

  
  cosdf = cosdf.sort_values(by=['cos_sim'],ascending = False)
  cosdf_top5 = cosdf[1:6]
  top5_df = metadata.loc[list(cosdf_top5['idx'])]
  top5_df.drop(['Unnamed: 0'],axis=1)
  

  return top5_df

In [ ]:
# melspectogram code
def melspectogram_code(signal, sr):
  hop_length = 512
  n_fft = 2048
  mel_signal = librosa.feature.melspectrogram(y=signal, sr=sr, hop_length=hop_length, n_fft=n_fft)
  spectrogram = np.abs(mel_signal)
  power_to_db = librosa.power_to_db(spectrogram, ref=np.max)
  return(power_to_db)

In [ ]:
def deep_learning_test(melspectogram):
  scaler = StandardScaler()
  temp = scaler.fit_transform(melspectogram)
  temp = cv2.resize(temp, (256, 256), interpolation = cv2.INTER_LANCZOS4)
  new2 = []
  new2.append(np.array([temp, temp, temp]))
  X_test = np.array(new2)
  X_test = np.swapaxes(np.swapaxes(X_test, 1, 2), 2, 3)
  testpred = []
  testprob = []
  for i in range(0, 5):
    xception = load_model('/content/drive/Shareddrives/2022-1 KUBIG 음악추천시스템/musicrecommendation_xception{zero}.h5'.format(zero = i))
    prob = xception.predict(X_test)
    testprob.append(prob)
    pred = np.argmax(prob, axis = 1)
    testpred.append(pred)
  pred1 = mode(testpred).mode
  pred1 = np.array(pred1).astype(int)

  return(pred1[0][0])




In [ ]:
def Reccomendation_system(title):
  global data

  if('Unnamed: 0' in data.columns):
    data = data.drop(['Unnamed: 0','genre_num'],axis=1)

  audio_file, save_file = mp3towav(title)
  y, sr = trim_audio_data(audio_file, save_file, title)
  melspectogram = melspectogram_code(y, sr)
  genre_num = deep_learning_test(melspectogram)

  new = extract_feature_means(y, sr, title)
  top5_df = rec_music(new,genre_num)
  top5_df = top5_df.drop(['Unnamed: 0','genre_num'],axis=1)

  print("Reccomending Top 5 ")

  return top5_df


In [ ]:
audio_file, save_file = mp3towav('414')
y, sr = trim_audio_data(audio_file, save_file)


Converting  414 .mp3 to wav...


TypeError: ignored

In [ ]:
Reccomendation_system('414')
#414: 임영웅 그대 나만 믿어요

Converting  414 .mp3 to wav...
Trimming  414 ...
Extracting Features of  414 ...
Reccomending Top 5 


,title,artist,release_date,genre
468,참 좋은 날,이찬원,2021.12.09,트로트
524,코로나19 이겨냅시다,태진아,2020.03.24,트로트
488,살아생전에,홍자,2019.06.07,트로트
460,물망초,정동원,2021.11.17,트로트
404,오라버니,금잔디,2013.03.01,트로트


In [ ]:
#박주영 - 이소라: 처음 느낌 그대로
Reccomendation_system('처음느낌그대로')

Converting  처음느낌그대로 .mp3 to wav...
Trimming  처음느낌그대로 ...
Extracting Features of  처음느낌그대로 ...
Reccomending Top 5 


,title,artist,release_date,genre
661,나를 좋아하지 않는 그대에게,박원,2015.11.12,발라드
613,거름,울랄라세션,2013.06.27,발라드
662,이럴거면 헤어지지 말았어야지,박원,2015.11.12,발라드
781,잠 못 드는 밤 비는 내리고,아이유(IU),2017.09.22,발라드
671,안아줘,정준일,2011.11.23,발라드


In [ ]:
#김유민 - 박재범(feat아이유): 가나다라
Reccomendation_system('가나다라')

Converting  가나다라 .mp3 to wav...
Trimming  가나다라 ...
Extracting Features of  가나다라 ...
Reccomending Top 5 


,title,artist,release_date,genre
173,SOJU (Feat. 2 Chainz),박재범,2018.05.25,힙합
41,BROKEN FERRARI,호미들,2022.03.08,힙합
177,Cereal (Feat. ZICO),Crush,2018.07.13,힙합
165,사라진 모든 것들에게(with ELLE KOREA),코드 쿤스트 (CODE KUNST),2020.11.16,힙합
21,이렇게 둘이,창모 (CHANGMO),2022.03.14,힙합


In [ ]:
#김태영 - 민호와 래퍼 기타등등: 쉬어
Reccomendation_system('쉬어')

Converting  쉬어 .mp3 to wav...
Trimming  쉬어 ...
Extracting Features of  쉬어 ...
Reccomending Top 5 


,title,artist,release_date,genre
69,수퍼비의 마세라티,베이식 (Basick),2022.03.28,힙합
71,RESPECT (Feat. GRAY & DJ Pumkin),로꼬,2015.07.14,힙합
188,DRIVE (Feat. GRAY),박재범,2016.10.20,힙합
17,우연히 봄,"로꼬, 유주 (YUJU)",2015.04.08,힙합
50,뚜두뚜두 (DDU-DU DDU-DU),BLACKPINK,2018.06.15,힙합


In [ ]:
#기다연 - 잔나비: 한밤의 뮤직
Reccomendation_system('한밤의뮤직')

Converting  한밤의뮤직 .mp3 to wav...
Trimming  한밤의뮤직 ...
Extracting Features of  한밤의뮤직 ...
Reccomending Top 5 


,title,artist,release_date,genre
836,다음에 봐,윤하(YOUNHA),2020.01.06,락
837,스무살 어느 날,윤하(YOUNHA),2020.01.06,락
972,위로,넬 (NELL),2021.09.02,락
846,처음엔 사랑이란게,버스커 버스커,2013.09.25,락
827,벚꽃 엔딩,버스커 버스커,2012.03.29,락


In [ ]:
#김혜림- 이소라: Track3
Reccomendation_system('Track3')

Converting  Track3 .mp3 to wav...
Trimming  Track3 ...
Extracting Features of  Track3 ...
Reccomending Top 5 


,title,artist,release_date,genre
632,사랑에 아파본 적 있나요,스탠딩에그,2018.09.07,발라드
602,All of My Life,박원,2017.07.27,발라드
728,눈,폴킴,2019.01.17,발라드
631,그때 헤어지면 돼,로이킴,2018.02.12,발라드
671,안아줘,정준일,2011.11.23,발라드


In [ ]:
'''
def show_top5(top5_df, title):

  print(title + '과 유사한 음악 \n' )
  for i in range(5):
    for j in range(5):
      print(top5_df.iloc[i][j])
'''
    

In [ ]:
'''
def trim_audio_data(raw_wav, trimmed_wav):

    sr = 44100
    sec = 30
    
    y, sr = librosa.load(raw_wav, sr = sr)
    ny = y[sr*60:sr*90]
    librosa.output.write_wav('cut_file.mp3', ny, sr) 
    sf.write(trimmed_wav, ny, sr)
    #print(save_file)

    return ny, sr
  '''

#wav_path = '/content/drive/Shareddrives/2022-1 KUBIG 음악추천시스템/'
#mp3_path = '/content/drive/Shareddrives/2022-1 KUBIG 음악추천시스템/지호/mp3'
#trim_path = '/content/drive/Shareddrives/2022-1 KUBIG 음악추천시스템/지호/trim'
#'C:/음악/trim'

In [ ]:
'''
def mp3towav(title):
  mp3_file = mp3_path + '/' + title +'.mp3'
  wav_file = wav_path + '/' + title + '.wav'
  subprocess.call(['ffmpeg', '-i', mp3_file, wav_file])
  return wav_file
'''



   # extract mfcc feature
    #mfcc_df = extract_mfcc_feature_means(audio_path,
    #                                y,
    #                                sample_rate=sr,
    #                                number_of_mfcc=number_of_mfcc)
